In [1]:
import pandas as pd
import glob
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.display import clear_output
import numpy as np
import re
import plotly.express as px
import plotly

In [3]:
pd.options.display.max_colwidth = 100

In [ ]:
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.dir", '.')
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.google-earth.kml+xml")

driver = webdriver.Firefox(firefox_profile=profile)
driver.get("http://dados.natal.br/dataset?q=rotas")

In [ ]:
enterprises = driver.find_elements_by_class_name("dataset-heading")
count=0
for i in range(len(enterprises)):
    enterprises = driver.find_elements_by_class_name("dataset-heading")
    enterprise = enterprises[count]
    enterprise.find_element_by_tag_name("a").click()
    links = driver.find_elements_by_class_name("dropdown")
    for link in links:
        link.click()
#         link.click()
        link.find_element_by_class_name("resource-url-analytics").click()
    driver.back()
    count+=1
    time.sleep(0.5)

In [ ]:
driver.close()

In [2]:
files = glob.glob('downloads/*')

lines_df = pd.read_csv("linhas_diretas_ufrn.csv")
lines_df.linha = lines_df.linha.str.replace("/","")

files_df = pd.DataFrame(files,columns={"arquivo"})
files_df["linha"] = files_df["arquivo"].str.extract("/([\w\d]+).")
files_df = files_df.sort_values(by="linha")

files_df = files_df.set_index("linha")
lines_df = lines_df.set_index("linha")
lines_df = lines_df.loc[list(np.invert(lines_df.index.duplicated()))]

In [3]:
display(lines_df)
display(files_df)

,nome
linha,
1029,"Nova Natal/Nova Descoberta, via Av. da Cheganç..."
18,Bairro Nordeste/Campus
30,"Felipe Camarão/Pirangi, via Candelária/Campus"
31,"Felipe Camarão/Pirangi, via Campus/Candelária"
48,"Santos Reis/Nova Descoberta, via Alecrim/Campus"
54A,"Ponta Negra/Campus, via Rua Walter Duarte Pereira"
57,"Mãe Luíza/Nova Descoberta, via Campus"
588,Circular UFRN (Inverso)
63,"Felipe Camarão/Campus, via Av. Bernardo Vieira"


,arquivo
linha,
01,downloads/01.kml
01a,downloads/01a.kml
02,downloads/02.kml
04,downloads/04.kml
04a,downloads/04a.kml
...,...
83a,downloads/83a.kml
84,downloads/84.kml
85,downloads/85.kml


In [4]:
[line for line in files_df.index if "a" in line]

['01a',
 '04a',
 '07a',
 '1029a',
 '13a',
 '20a',
 '21a',
 '27a',
 '3101a',
 '33a',
 '33a',
 '36a',
 '41a',
 '41a',
 '588a',
 '599a',
 '63a',
 '64a43',
 '75a',
 '78a47',
 '83a']

In [5]:
def getArquivos(linha):
    l=[]
    if (linha in files_df.index):
        if isinstance(files_df.loc[linha].arquivo,str):
            l+= [files_df.loc[linha].arquivo]
        else:
            l+= files_df.loc[linha].arquivo.tolist()
    if (str(linha)+"a" in files_df.index):
        if isinstance(files_df.loc[str(linha)+"a"].arquivo,str):
            l+= [files_df.loc[str(linha)+"a"].arquivo]
        else:
            l+= files_df.loc[str(linha)+"a"].arquivo.tolist()
    return l

In [6]:
lines_df.index.map(getArquivos)

Index([                     ['downloads/1029.kml', 'downloads/1029(1).kml', 'downloads/1029a.kml'],
                                                                              ['downloads/18.kml'],
                                                                              ['downloads/30.kml'],
                                                                              ['downloads/31.kml'],
                                                                              ['downloads/48.kml'],
                                                                                                [],
                                                                              ['downloads/57.kml'],
       ['downloads/588(2).kml', 'downloads/588.kml', 'downloads/588(1).kml', 'downloads/588a.kml'],
                                                         ['downloads/63.kml', 'downloads/63a.kml'],
                                                                              ['downloads/66.kml']],

In [7]:
[files_df.loc["54"].arquivo]

['downloads/54.kml']

In [8]:
lines_df["arquivo"]=lines_df.index.map(getArquivos)
lines_df.loc["54A"].arquivo.append('downloads/54.kml')
lines_df

,nome,arquivo
linha,,
1029,"Nova Natal/Nova Descoberta, via Av. da Cheganç...","[downloads/1029.kml, downloads/1029(1).kml, do..."
18,Bairro Nordeste/Campus,[downloads/18.kml]
30,"Felipe Camarão/Pirangi, via Candelária/Campus",[downloads/30.kml]
31,"Felipe Camarão/Pirangi, via Campus/Candelária",[downloads/31.kml]
48,"Santos Reis/Nova Descoberta, via Alecrim/Campus",[downloads/48.kml]
54A,"Ponta Negra/Campus, via Rua Walter Duarte Pereira",[downloads/54.kml]
57,"Mãe Luíza/Nova Descoberta, via Campus",[downloads/57.kml]
588,Circular UFRN (Inverso),"[downloads/588(2).kml, downloads/588.kml, down..."
63,"Felipe Camarão/Campus, via Av. Bernardo Vieira","[downloads/63.kml, downloads/63a.kml]"


In [9]:
%%time
points=[]
filenames = []

for arquivos in lines_df.arquivo:
    filenames+=arquivos

for filename in filenames:
    file = open(filename,mode='r')
    file_text = file.read()
    file.close()
    points+=[filename+","+point for point in re.findall("<Point><altitudeMode>clampToGround<\/altitudeMode><coordinates>([^>][\w\d\s\,\.-]+[^<])<\/coordinates><\/Point>",file_text)]
points[0:10]

CPU times: user 26.7 ms, sys: 10.7 ms, total: 37.4 ms
Wall time: 269 ms


['downloads/1029.kml,-35.263028908520937,-5.724837854504585,48.68',
 'downloads/1029.kml,-35.263291010633111,-5.725240102037787',
 'downloads/1029.kml,-35.263340380042791,-5.725347893312573',
 'downloads/1029.kml,-35.263314060866833,-5.725434143096209,49.159999999999997',
 'downloads/1029.kml,-35.263195121660829,-5.725563140586019,50.130000000000003',
 'downloads/1029.kml,-35.263029076159,-5.725678391754627,50.130000000000003',
 'downloads/1029.kml,-35.262844925746322,-5.725787859410048,50.130000000000003',
 'downloads/1029.kml,-35.26268701069057,-5.725883161649108,50.609999999999999',
 'downloads/1029.kml,-35.262505793944001,-5.726025905460119,50.130000000000003',
 'downloads/1029.kml,-35.262633534148335,-5.726236710324884,50.130000000000003']

In [29]:
def getLineFromFilename(filename):
    for i in range(len(lines_df)):
        if filename in lines_df.iloc[i].arquivo:
            return lines_df.index[i]
def getLineNameFromFilename(filename):
    for i in range(len(lines_df)):
        if filename in lines_df.iloc[i].arquivo:
            return lines_df.iloc[i].nome

In [30]:
points_df = pd.DataFrame([point.split(",")[0:3] for point in points],columns=["arquivo","long","lat"])
points_df["linha"] = points_df.arquivo.map(getLineFromFilename)
points_df["nome"] = points_df.arquivo.map(getLineNameFromFilename)
points_df.long = points_df.long.astype(float)
points_df.lat = points_df.lat.astype(float)
points_df

,arquivo,long,lat,linha,nome
0,downloads/1029.kml,-35.263029,-5.724838,1029,"Nova Natal/Nova Descoberta, via Av. da Cheganç..."
1,downloads/1029.kml,-35.263291,-5.725240,1029,"Nova Natal/Nova Descoberta, via Av. da Cheganç..."
2,downloads/1029.kml,-35.263340,-5.725348,1029,"Nova Natal/Nova Descoberta, via Av. da Cheganç..."
3,downloads/1029.kml,-35.263314,-5.725434,1029,"Nova Natal/Nova Descoberta, via Av. da Cheganç..."
4,downloads/1029.kml,-35.263195,-5.725563,1029,"Nova Natal/Nova Descoberta, via Av. da Cheganç..."
...,...,...,...,...,...
6098,downloads/66.kml,-35.169165,-5.885262,66,"Ponta Negra/Cidade da Esperança, via Bom Pastor"
6099,downloads/66.kml,-35.168969,-5.885481,66,"Ponta Negra/Cidade da Esperança, via Bom Pastor"
6100,downloads/66.kml,-35.168676,-5.885688,66,"Ponta Negra/Cidade da Esperança, via Bom Pastor"
6101,downloads/66.kml,-35.168585,-5.885600,66,"Ponta Negra/Cidade da Esperança, via Bom Pastor"


In [31]:
points_df.to_csv("points_df.csv",index=False)

In [12]:
px.set_mapbox_access_token("")
fig = px.scatter_mapbox(points_df, lat="lat", lon="long",color="nome",size_max=15, zoom=10)
fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'accesstoken' property of layout.mapbox
        Received value: ''

    The 'accesstoken' property is a string and must be specified as:
      - A non-empty string